In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import json
sys.path.append("..")
from psflearning.psflearninglib import psflearninglib
with open('datadir.json','r') as file:
    maindatadir = json.load(file)['main_data_dir']

In [ ]:
#load parameters
paramfile = 'params_default.json'
L = psflearninglib()
L.getparam(paramfile)

In [ ]:
# load data
L.param['datapath'] = maindatadir + r'bead data\01-08-2022 bead\100nm_600/'
L.param['keyword'] = 'bead'
L.param['subfolder'] = 'bead'
L.param['format'] = '.mat'
L.param['channeltype'] = '4pi'
L.param['gain'] = 1/2.27
L.param['ccd_offset'] = 100
L.param['stage_mov_dir'] = 'reverse'
images = L.load_data()


In [ ]:
# setup PSF type
L.param['PSFtype'] = 'voxel'
L.getpsfclass()

In [ ]:
# crop bead
L.param['pixelsize_x'] =  0.129
L.param['pixelsize_y'] = 0.129
L.param['pixelsize_z'] = 0.05
L.param['roi_size'] = [21,21]
L.param['plotall'] = True
L.param['max_bead_number'] = 50
L.param['bead_radius'] = 0.05
L.param['modulation_period'] = 0.29 # unit: micron, 0.26 for emission filter at 600 nm, 0.29 for emission filter at 676 nm
L.param['FOV']['z_start'] = 0
dataobj = L.prep_data(images)

In [ ]:
# learn PSF
L.param['iteration'] = 100
L.param['estimate_drift'] = False
L.param['option_params']['phase_dm'] = [2,0,-2]
psfobj,fitter = L.learn_psf(dataobj,time=0)

In [ ]:
# save as .h5 file
L.param['savename'] = L.param['datapath'] + 'psfmodel'
resfile, res_dict, loc_dict, rois_dict = L.save_result(psfobj,dataobj,fitter)

In [ ]:
# show estimated bead positions, photon and background
Nchannel = rois_dict['psf_data'].shape[0]
cor = rois_dict['cor'][0]
ref_pos1 = res_dict['channel0']['pos']
fig = plt.figure(figsize=[16,8])
ax = fig.add_subplot(2,4,1)
plt.plot(ref_pos1[:,1]-cor[:,0])
plt.title('y')
ax = fig.add_subplot(2,4,2)
plt.plot(ref_pos1[:,2]-cor[:,1])
plt.title('x')
ax = fig.add_subplot(2,4,3)
plt.plot(ref_pos1[:,0])
plt.title('z')
ax = fig.add_subplot(2,4,4)
plt.plot(np.angle(res_dict['channel0']['intensity']))
ax.set_title('phi')
ax = fig.add_subplot(2,4,5)
plt.plot(np.abs(res_dict['channel0']['intensity']))
ax.set_title('photon')
ax = fig.add_subplot(2,4,6)
for i in range(Nchannel):
    plt.plot(res_dict['channel'+str(i)]['bg'])
plt.title('background')
ax = fig.add_subplot(2,4,7)
plt.plot(res_dict['channel0']['drift_rate'])
ax.set_title('drift rate')

In [ ]:
# compare PSF model and data
ind = 0 # bead index
psf_data = rois_dict['psf_data']
psf_fit = rois_dict['psf_fit']

Nchannel = psf_data.shape[0]

for i in range(0,Nchannel):
    im1 = psf_data[i,ind,0]
    im2 = psf_fit[i,ind,0]
    Nz = im1.shape[0]
    zind = range(1,Nz,4)
    fig = plt.figure(figsize=[3*len(zind),6])
    for i,id in enumerate(zind):
        ax = fig.add_subplot(2,len(zind),i+1)
        plt.imshow(im1[id],cmap='twilight')
        plt.axis('off')
        ax = fig.add_subplot(2,len(zind),i+1+len(zind))
        plt.imshow(im2[id],cmap='twilight')
        plt.axis('off')


In [ ]:
# show localization result
Nz = loc_dict['loc']['z'].shape[1]
fig = plt.figure(figsize=[16,4])
ax = fig.add_subplot(1,3,1)
plt.plot(loc_dict['loc']['x'].transpose()*L.param['pixelsize_x']*1e3,'k',alpha=0.1)
plt.plot(loc_dict['loc']['x'][0]*0.0,'r')
ax.set_ylabel('x bias (nm)')
ax = fig.add_subplot(1,3,2)
plt.plot(loc_dict['loc']['y'].transpose()*L.param['pixelsize_y']*1e3,'k',alpha=0.1)
plt.plot(loc_dict['loc']['x'][0]*0.0,'r')
ax.set_ylabel('y bias (nm)')
ax = fig.add_subplot(1,3,3)
plt.plot(np.transpose(loc_dict['loc']['z']-np.linspace(0,Nz-1,Nz))*L.param['pixelsize_z']*1e3,'k',alpha=0.1)
plt.plot(loc_dict['loc']['x'][0]*0.0,'r')
ax.set_ylabel('z bias (nm)')
ax.set_ylim([-40,40])

In [ ]:
# show transformation
np.set_printoptions(precision=4,suppress=True)
print(res_dict['T'])
ref_pos = res_dict['channel0']['pos']
dxy = res_dict['xyshift'] 
fig = plt.figure(figsize=[4*Nchannel,4])

for i in range(0,Nchannel):
    pos = res_dict['channel'+str(i)]['pos']
    ax = fig.add_subplot(1,Nchannel,i+1)
    plt.plot(ref_pos[:,1],ref_pos[:,2],'.')
    plt.plot(pos[:,1]-dxy[i][0],pos[:,2]-dxy[i][1],'x')
    plt.plot(fitter.psf.imgcenter[0],fitter.psf.imgcenter[1],'o')
